In [4]:
import pandas as pd

from pathlib import Path

from skimage.feature import hog
from skimage.io import imread
from skimage.io import imshow
from skimage.transform import resize

In [5]:
breeds: list[tuple[str, str]] = [
    ("dog", "chihuahua"),
    ("dog", "saint_bernard"),
    ("cat", "siamese"),
    ("cat", "sphynx"),
]

In [ ]:
size: int = 128
quality: int = 16

metadata: dict[str, list] = {
    "animal": list(),
    "breed": list(),
    "filename": list(),
}

features: list = list()

for animal, breed in breeds:
    files: Path = Path(f"../images/{breed}").resolve()
    
    for file in files.iterdir():

        image = imread(file.as_posix())

        resized_image = resize(image, (size, size))

        hog_feature, _ = hog(
            image=resized_image,
            orientations=9,
            pixels_per_cell=(quality, quality),
            cells_per_block=(2, 2),
            visualize=True,
            channel_axis=1,
        )

        features.append(hog_feature)

        metadata["animal"].append(animal)
        metadata["breed"].append(breed)
        metadata["filename"].append(file.name)

features_df: pd.DataFrame = pd.DataFrame(features)
features_df.rename(lambda x: f"feature_{int(x) + 1}", axis="columns")

metadata_df: pd.DataFrame = pd.DataFrame.from_dict(metadata)

df: pd.DataFrame = pd.concat([metadata_df, features_df], axis="columns")

output_file: Path = Path(f"../outputs/hog_features_{size}px_x_{size}px_{quality}px.csv").resolve()
output_file.parent.mkdir(parents=True, exist_ok=True)

df.to_csv(output_file.as_posix(), index=False)

OSError: Cannot save file into a non-existent directory: '/home/rary/ufrn/ml/image-extraction-knn/outputs'